This notebook is for the final project of NLP (Fall 2023) - Tashi and Jungi @ SNU CSE

# Skill Extraction from Job Posting via LLM Annotations  
Refer to the slides for a intuitive understanding --> 

### Dependecies

Fork prodigy-openai-recpies repo and use it as the root

In [314]:
import pandas as pd
import spacy
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

## Developing the Sentence Classifier

In [97]:
df = pd.read_csv('/Users/tashi/Desktop/Fall/Thesis/src/rezi/data/allJobs.csv')

In [98]:
df.head()

,_id,url,job_title,company_name,workplace_type,department,education_level,employment_type,max_compensation,role_requirement,...,updated_at,crawled_at,published_at,location,geo_lat,geo_lon,uid,user,country_code,work_group
0,1uHjDYcBzdM5htbMbZyl,https://jobs.lever.co/ppwi/3444b579-5abf-46b6-...,Reproductive Healthcare Assistant,"Planned Parenthood of Wisconsin, Inc.",ON-SITE,CLINICALMEDICAL,NaN,FULL TIME,NaN,Essential Duties and Responsibilities:\nInclud...,...,NaN,2023-04-18T05:23:39+00:00,NaN,"Milwaukee, Wisconsin, United States",43.038314,-87.907798,NaN,NaN,US,NaN
1,rzpgL4UBZiaMfNUIY5ZU,https://jobs.lever.co/viseven/186ba6c7-d790-48...,Senior Account Manager,Viseven,HYBRID,VISEVEN – EUROPE,NaN,FULL-TIME,NaN,Responsibilities:\nOversee client and account ...,...,NaN,2023-04-18T05:23:39+00:00,NaN,"Poznań, Greater Poland Voivodeship, Poland",52.400663,16.919733,NaN,NaN,PL,NaN
2,Ek9-D4cB8T_Z8WytG2FR,https://jobs.lever.co/airwallex/e3261dc7-ca5f-...,"Director, Revenue Strategy",Airwallex,HYBRID,"PLATFORM BUSINESS – REVENUE STRATEGY & OPS, PL...",NaN,FULL-TIME,NaN,Drive global Monetisation strategy\nDesign end...,...,NaN,2023-04-18T05:23:53+00:00,NaN,"San Francisco, California, United States",37.779026,-122.419906,NaN,NaN,US,NaN
3,9U94NIUB8T_Z8Wytiz9U,https://jobs.lever.co/upstox/f186fca4-3d77-495...,Video Editor,Upstox,HYBRID,DEPARTMENT – MARKETING,NaN,FULL-TIME,NaN,NaN,...,NaN,2023-04-18T05:23:53+00:00,NaN,"Mumbai, Maharashtra, India",19.078545,72.878176,NaN,NaN,IN,NaN
4,dU9cEIcB8T_Z8Wytj2eD,https://jobs.lever.co/accolitedigital/76f71cf6...,Sr Project Manager - (Data Center and Cloud Mi...,Accolite Digital,HYBRID,ACCOLITE – IT,NaN,CONTRACT,NaN,"Responsibilities:\nDirects large-scale, comple...",...,NaN,2023-04-18T05:23:54+00:00,NaN,"Phoenix, Arizona, United States",33.448437,-112.074141,NaN,NaN,US,NaN


Remove Duplicates and NaNs

In [99]:
df = df.drop_duplicates(subset=['role_requirement'], keep='first')
df.dropna(subset=['role_requirement'], inplace=True)
df = df.reset_index(drop=True)

In [100]:
df.head()

,_id,url,job_title,company_name,workplace_type,department,education_level,employment_type,max_compensation,role_requirement,...,updated_at,crawled_at,published_at,location,geo_lat,geo_lon,uid,user,country_code,work_group
0,1uHjDYcBzdM5htbMbZyl,https://jobs.lever.co/ppwi/3444b579-5abf-46b6-...,Reproductive Healthcare Assistant,"Planned Parenthood of Wisconsin, Inc.",ON-SITE,CLINICALMEDICAL,NaN,FULL TIME,NaN,Essential Duties and Responsibilities:\nInclud...,...,NaN,2023-04-18T05:23:39+00:00,NaN,"Milwaukee, Wisconsin, United States",43.038314,-87.907798,NaN,NaN,US,NaN
1,rzpgL4UBZiaMfNUIY5ZU,https://jobs.lever.co/viseven/186ba6c7-d790-48...,Senior Account Manager,Viseven,HYBRID,VISEVEN – EUROPE,NaN,FULL-TIME,NaN,Responsibilities:\nOversee client and account ...,...,NaN,2023-04-18T05:23:39+00:00,NaN,"Poznań, Greater Poland Voivodeship, Poland",52.400663,16.919733,NaN,NaN,PL,NaN
2,Ek9-D4cB8T_Z8WytG2FR,https://jobs.lever.co/airwallex/e3261dc7-ca5f-...,"Director, Revenue Strategy",Airwallex,HYBRID,"PLATFORM BUSINESS – REVENUE STRATEGY & OPS, PL...",NaN,FULL-TIME,NaN,Drive global Monetisation strategy\nDesign end...,...,NaN,2023-04-18T05:23:53+00:00,NaN,"San Francisco, California, United States",37.779026,-122.419906,NaN,NaN,US,NaN
3,dU9cEIcB8T_Z8Wytj2eD,https://jobs.lever.co/accolitedigital/76f71cf6...,Sr Project Manager - (Data Center and Cloud Mi...,Accolite Digital,HYBRID,ACCOLITE – IT,NaN,CONTRACT,NaN,"Responsibilities:\nDirects large-scale, comple...",...,NaN,2023-04-18T05:23:54+00:00,NaN,"Phoenix, Arizona, United States",33.448437,-112.074141,NaN,NaN,US,NaN
4,p-GmEIcBzdM5htbMf7AX,https://jobs.lever.co/varicent/0469919c-02c1-4...,Business Development Representative - Remote,Varicent,REMOTE,SALES,NaN,FULL TIME,NaN,What you will do:\nYou will engage C-level dec...,...,NaN,2023-04-18T05:24:00+00:00,NaN,"Chicago, Illinois, United States",41.875562,-87.624421,NaN,NaN,US,NaN


In [101]:
df = df[['job_title', 'role_requirement']]
df.rename(columns={'role_requirement': 'text'}, inplace=True)

In [102]:
df.head()

,job_title,text
0,Reproductive Healthcare Assistant,Essential Duties and Responsibilities:\nInclud...
1,Senior Account Manager,Responsibilities:\nOversee client and account ...
2,"Director, Revenue Strategy",Drive global Monetisation strategy\nDesign end...
3,Sr Project Manager - (Data Center and Cloud Mi...,"Responsibilities:\nDirects large-scale, comple..."
4,Business Development Representative - Remote,What you will do:\nYou will engage C-level dec...


Randomly choose 200 jobs for sentnece labelling

In [103]:
sample_df = df.sample(n=200, random_state=42)

In [104]:
sample_df

,job_title,text
3355,-Enterprise Account Executive,What you will do\nDevelop an Enterprise Accoun...
2405,"Distribution Center (DC) Supervisor, 2nd Shift",Responsibilities\nSupervise and assist in dail...
4373,"Technical Project Manager, Noida",Skills & Knowledge\nStrong Project Management ...
107,Retail Category Leaders - Electronics Category,Your Main Responsibilities\nDeveloping busines...
9391,Senior Engine Programmer - Network,"Responsibilities\nTake ownership, develop and ..."
...,...,...
259,Fleet Advisor - Austin,Outcomes\nMachine listings - We’re moving iron...
6429,Manufacturing Engineer,What You'll Do:\nImplement a custom Manufactur...
5406,Commercial Executive – Licensing,WHAT YOU'LL BRING\nBachelor's degree in busine...
1575,Enterprise Solutions Engineer - Greater DC Area,Responsibilities\nBe the subject matter author...


Split the job postings by "\n" to prepare for GPT to label the sentences

In [105]:
sample_df['text'] = sample_df['text'].str.split(r'\n')  # Splitting by \n
exploded_df = sample_df.explode('text')
# Remove any trailing periods and whitespace
exploded_df['text'] = exploded_df['text'].str.strip('.').str.strip()

In [106]:
exploded_df

,job_title,text
3355,-Enterprise Account Executive,What you will do
3355,-Enterprise Account Executive,Develop an Enterprise Account Plan for strateg...
3355,-Enterprise Account Executive,Accurately forecast sales revenue and pipeline...
3355,-Enterprise Account Executive,Engage with Fortune 1000 accounts across a var...
3355,-Enterprise Account Executive,Attend relevant trade shows when required
...,...,...
80,Environment Modeler Senior,Extensive experience with SpeedTree
80,Environment Modeler Senior,Extensive experience with Arnold render
80,Environment Modeler Senior,Experience with Houdini is a plus
80,Environment Modeler Senior,"Experience with traditional drawing, and/or pa..."


In [107]:
exploded_df.to_json('/Users/tashi/Desktop/Fall/Thesis/src/nlp/data/sentence_level/split_sents.jsonl', orient='records', lines=True)

**Here we use spacy and prodigy from Explosion.ai repo https://github.com/explosion/prodigy-openai-recipes to label each sentence using GPT**

the --labels argument is defined in ./prodigy-openai-recipes/templates/textcat_prompt.jinja2

In [108]:
!python3 -m prodigy textcat.openai.fetch ../data/sentence_level/split_sents.jsonl ../data/sentence_level/sents_predictions.jsonl --e --labels section_title,company_info,skills,miscellaneous -F ./recipes/openai_textcat.py

4129it [09:59,  6.89it/s]


GPT labeled sentences are saved to ./data/sentence_level/sent_cls_predictions.jsonl

**For GPU compute power, we train/finetune bert using this acquired labels in this colab notebook:**    
https://colab.research.google.com/drive/13HwU4A-Ir5pnRPSrUxqSO2P7D87oJ6Nl#scrollTo=OjAX-Q7CMv8Z

**Sentence Classifer is saved to ../nlp/models/sent_cls**

## Developing the NER Model

In [111]:
df2 = pd.read_json('/Users/tashi/Desktop/Fall/Thesis/src/nlp/data/sentence_level/sents_predictions.jsonl', lines=True)

In [112]:
df2['accept'] = df2['accept'].apply(lambda x: x[0])

In [113]:
df2 = df2[['text', 'accept']]

In [120]:
df2.head()

,text,accept
0,What you will do,section_title
1,Develop an Enterprise Account Plan for strateg...,skills
2,Accurately forecast sales revenue and pipeline...,miscellaneous
3,Engage with Fortune 1000 accounts across a var...,company_info
4,Attend relevant trade shows when required,miscellaneous


Keep only the sentences labelled as skills

In [116]:
fitlered_df2 = df2[df2['accept'] == 'skills']

In [122]:
print(fitlered_df2.shape)
fitlered_df2.head()

(2723, 2)


,text,accept
1,Develop an Enterprise Account Plan for strateg...,skills
5,Manage the entire sales process from prospect ...,skills
8,Maintain Salesforce.com CRM and ensure a stron...,skills
9,5+ years of SaaS enterprise sales experience,skills
10,Expertise in managing multi-stakeholder sales ...,skills


In [123]:
fitlered_df2.to_json('/Users/tashi/Desktop/Fall/Thesis/src/nlp/data/token_level/skills_sents.jsonl', orient='records', lines=True)

Again we turn to spacy and prodigy to help us label the hard and soft skills from each sentence marked as "skills"

In [124]:
!python3 -m prodigy ner.openai.fetch /Users/tashi/Desktop/Fall/Thesis/src/nlp/data/token_level/skills_sents.jsonl /Users/tashi/Desktop/Fall/Thesis/src/nlp/data/token_level/ner_predictions.jsonl "hard_skill,soft_skill" -F ./recipes/openai_ner.py

100%|███████████████████████████████████████| 2723/2723 [05:55<00:00,  7.65it/s]


We import the .jsonl file containing predictions into prodigy's db for easy manipulation

In [125]:
!python3 -m prodigy db-in ner_preds /Users/tashi/Desktop/Fall/Thesis/src/nlp/data/token_level/ner_predictions.jsonl         

✔ Created dataset 'ner_preds' in database SQLite
✔ Imported 2723 annotated examples and saved them to 'ner_preds'
(session 2023-12-13_16-27-17) in database SQLite
Found and keeping existing "answer" in 0 examples


Load the labelled dataset

In [127]:
ner_df = pd.read_json("/Users/tashi/Desktop/Fall/Thesis/src/nlp/data/token_level/ner_predictions.jsonl", lines=True)

In [128]:
ner_df.head()

,text,accept,openai,tokens,meta,spans,_input_hash,_task_hash
0,Develop an Enterprise Account Plan for strateg...,skills,{'prompt': 'The following are the definitions ...,"[{'text': 'Develop', 'start': 0, 'end': 7, 'id...",{},"[{'label': 'hard_skill', 'start': 0, 'end': 7,...",815660824,1280380821
1,Manage the entire sales process from prospect ...,skills,{'prompt': 'The following are the definitions ...,"[{'text': 'Manage', 'start': 0, 'end': 6, 'id'...",{},[],-1688214404,1228344610
2,Maintain Salesforce.com CRM and ensure a stron...,skills,{'prompt': 'The following are the definitions ...,"[{'text': 'Maintain', 'start': 0, 'end': 8, 'i...",{},"[{'label': 'hard_skill', 'start': 0, 'end': 27...",1120410658,-901354282
3,5+ years of SaaS enterprise sales experience,skills,{'prompt': 'The following are the definitions ...,"[{'text': '5', 'start': 0, 'end': 1, 'id': 0, ...",{},"[{'label': 'hard_skill', 'start': 12, 'end': 4...",-1878743098,1520159998
4,Expertise in managing multi-stakeholder sales ...,skills,{'prompt': 'The following are the definitions ...,"[{'text': 'Expertise', 'start': 0, 'end': 9, '...",{},"[{'label': 'soft_skill', 'start': 0, 'end': 9,...",2074544853,1492035611


Split 80/20

In [141]:
!python3 -m prodigy data-to-spacy ./data/annotations/ --ner ner_preds -es 0.2

✔ Created output directory
ℹ Using language 'en'

============================== Generating data ==============================
Components: ner
Merging training and evaluation data for 1 components
  - [ner] Training: 2189 | Evaluation: 534 (20% split)
Training: 2089 | Evaluation: 522
Labels: ner (2)
✔ Saved 2089 training examples
data/annotations/train.spacy
✔ Saved 522 evaluation examples
data/annotations/dev.spacy

============================= Generating config =============================
ℹ Auto-generating config with spaCy
✔ Generated training config

======================== Generating cached label data ========================
✔ Saving label data for component 'ner'
data/annotations/labels/ner.json

============================= Finalizing export =============================
✔ Saved training config
data/annotations/config.cfg

To use this data for training with spaCy, you can run:
python -m spacy train data/annotations/config.cfg --paths.train data/annotations/train.spacy --p

Again we finetune using the obtained NER Labels in colab notebook:  
https://colab.research.google.com/drive/13HwU4A-Ir5pnRPSrUxqSO2P7D87oJ6Nl#scrollTo=OjAX-Q7CMv8Z

## Inference Pipeline

### Sentence Classification

In [230]:
# Load the trained SpaCy model
model_path = "/Users/tashi/Desktop/Fall/Thesis/src/nlp/models/sent_cls_model/model-last"
nlp = spacy.load(model_path)

In [240]:
def sent_cls(text):
    sents = text.split('\n')
    skill_sents = []
    for sent in sents:
        if len(sent) <= 0:
            continue
        doc = nlp(sent)
        max_label = max(doc.cats, key=lambda label: doc.cats[label])
        max_score = doc.cats[max_label]
        if max_label == 'skills':
            skill_sents.append(sent)
        # Print the text, label, and its score
        print(f"Text: '{sent}'\nPredicted category: {max_label} with score {max_score}\n")
    return skill_sents


### NER inference

#### Initializing and post processing Steps

In [315]:
# Load the model and tokenizer outside the function
model_path = "/Users/tashi/Desktop/Fall/Thesis/src/nlp/models/bert-ner-model"
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()  # Set the model to evaluation mode

# Define the NER tags
tag_names = ['O', 'B-HardSkill', 'B-SoftSkill', 'I-SoftSkill', 'I-HardSkill']  # Adjust based on your model

def perform_ner_inference(text, model, tokenizer, tag_names):
    """
    Function to perform NER inference on a single piece of text.

    Args:
    text (str): The text for NER inference.
    model: The loaded NER model.
    tokenizer: The tokenizer for preprocessing text.
    tag_names (list): List of NER tag names used in the model.

    Returns:
    list: List of tuples (word, tag) with NER tags for each word in the text.
    """
    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt", is_split_into_words=False)

    # Move model and inputs to the same device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Process logits and get predicted tags
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2)
    predicted_tags = [tag_names[tag_id] for tag_id in predictions[0].cpu().numpy()]

    # Pair tokens with their predicted tags
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0].cpu().numpy())
    token_tag_pairs = list(zip(tokens, predicted_tags))

    aligned_tags = []
    current_word = ""
    current_tag = None

    for token, tag in zip(tokens, predicted_tags):
        if token.startswith("##"):
            current_word += token[2:]  # Remove '##' and concatenate
        elif current_word:
            # Append the previous word and its tag
            aligned_tags.append((current_word, current_tag))
            current_word, current_tag = token, tag
        else:
            current_word, current_tag = token, tag

        # Skip special tokens
        if token in tokenizer.all_special_tokens:
            continue

    # Append the last word if it exists
    if current_word:
        aligned_tags.append((current_word, current_tag))

    return aligned_tags


Obtain raw NER results

In [316]:
def get_raw_results(sents):
    results = []
    for sent in sents:
        ner_results = perform_ner_inference(sent, model, tokenizer, tag_names)
        results.append(ner_results)
        print(f"Text: {sent}\nNER Results: {ner_results}\n")
    return results

Get the neccessary entities only i.e., remove unmarked entitites

In [317]:
def get_entities(aligned_tags):
    """
    Function to extract entities from a list of aligned tags.

    Args:
    aligned_tags (list): List of tuples (word, tag) with NER tags for each word in the text.

    Returns:
    list: List of tuples (entity, tag) with extracted entities.
    """
    final_entities = []
    current_entity = []
    current_entity_tag = None

    for tups in aligned_tags:
        for word, tag in tups:
            if tag.startswith("B-"):
                if current_entity:
                    # Combine the words in the current entity and add to final list
                    entity = " ".join(current_entity)
                    final_entities.append((entity, current_entity_tag))
                    current_entity = [word]
                    current_entity_tag = tag
                else:
                    current_entity.append(word)
                    current_entity_tag = tag
            elif tag.startswith("I-") and current_entity_tag is not None:
                # Continue the current entity
                current_entity.append(word)
            else:
                if current_entity:
                    # Finish the current entity and start a new one
                    entity = " ".join(current_entity)
                    final_entities.append((entity, current_entity_tag))
                    current_entity = []
                    current_entity_tag = None

        # Append the last entity if it exists
        if current_entity:
            entity = " ".join(current_entity)
            final_entities.append((entity, current_entity_tag))

    return final_entities

Categorize by hard and soft skillls

In [318]:
def categorize_skills(entities_list):
    hard_skills = []
    soft_skills = []
    for entity, tag in entities_list:
        if 'HardSkill' in tag:
            hard_skills.append(entity)
        elif 'SoftSkill' in tag:
            soft_skills.append(entity)

    return hard_skills, soft_skills


## Testing inference on an example

Copy & paste a job description in the following:

In [323]:
text = """Responsibilities include:
Designing and developing client-side features that solve market problems within project schedule deadlines.

Familiarity of emerging technologies in web-based applications

Ensuring high quality of the product through comprehensive unit, component, and integration tests

Collaborating with quality assurance engineers to enable thorough end to end system testing.
Working with senior engineers and other stakeholders to clarify requirements and review software design proposals.

Contributing to organization culture, disciplines, process & tools


QUALIFICATIONS:

A completed bachelor’s degree in Computer Science, a related technical field, or equivalent practical experience.
Relevant project or internship experience in software development, particularly in web-based UI applications.
Working knowledge of JavaScript, HTML, CSS, and Java.
Experience with REST APIs and building UI applications using them.
A self-starter attitude with the initiative to get things done.


You'll excite us if you have:


Familiarity with Angular 2+, NodeJS, and ngrx state management.
Familiarity with Java and Spring framework.
Willingness to learn and adapt to new technologies.
Experience with browser-based automation frameworks such as Cypress or Selenium is a plus.
Familiarity with Linux.
Excellent communication and interpersonal skills.

Our salary ranges consider various factors, including but not limited to benchmarking by independent third-party consultants, skills, years of experience, training, education, geography, and other business needs. 
"""

**Classify each sentences and keep only the sentences marked as "skills"**

In [329]:
sents = sent_cls(text)

Text: 'Responsibilities include:'
Predicted category: miscellaneous with score 0.6285697817802429

Text: 'Designing and developing client-side features that solve market problems within project schedule deadlines.'
Predicted category: miscellaneous with score 0.8096529245376587

Text: 'Familiarity of emerging technologies in web-based applications'
Predicted category: skills with score 0.6386134624481201

Text: 'Ensuring high quality of the product through comprehensive unit, component, and integration tests'
Predicted category: skills with score 0.8064651489257812

Text: 'Collaborating with quality assurance engineers to enable thorough end to end system testing.'
Predicted category: miscellaneous with score 0.9343215823173523

Text: 'Working with senior engineers and other stakeholders to clarify requirements and review software design proposals.'
Predicted category: miscellaneous with score 0.8439223170280457

Text: 'Contributing to organization culture, disciplines, process & tools

In [330]:
aligned_ents = get_raw_results(sents)

Text: Familiarity of emerging technologies in web-based applications
NER Results: [('[CLS]', 'O'), ('Familiarity', 'B-HardSkill'), ('of', 'I-HardSkill'), ('emerging', 'I-HardSkill'), ('technologies', 'I-HardSkill'), ('in', 'O'), ('web', 'B-HardSkill'), ('-', 'I-HardSkill'), ('based', 'I-HardSkill'), ('applications', 'I-HardSkill'), ('[SEP]', 'B-HardSkill')]

Text: Ensuring high quality of the product through comprehensive unit, component, and integration tests
NER Results: [('[CLS]', 'O'), ('Ensuring', 'O'), ('high', 'O'), ('quality', 'O'), ('of', 'O'), ('the', 'O'), ('product', 'O'), ('through', 'O'), ('comprehensive', 'B-HardSkill'), ('unit', 'I-HardSkill'), (',', 'O'), ('component', 'B-HardSkill'), (',', 'O'), ('and', 'O'), ('integration', 'B-HardSkill'), ('tests', 'I-HardSkill'), ('[SEP]', 'O')]

Text: Relevant project or internship experience in software development, particularly in web-based UI applications.
NER Results: [('[CLS]', 'O'), ('Relevant', 'O'), ('project', 'O'), ('or'

In [326]:
ents = get_entities(aligned_ents)

In [327]:
h_skill, s_skill = categorize_skills(ents)

**Final Result**

In [328]:
print(f"Hard Skills: {h_skill}\n\nSoft Skills: {s_skill}")

Hard Skills: ['Familiarity of emerging technologies', 'web - based applications', '[SEP]', '[SEP]', 'comprehensive unit', 'component', 'integration tests', 'software development', 'web - based UI applications', 'JavaScript', 'HTML', 'CSS', 'Java', 'building UI applications', 'Familiarity with', 'Angular 2 +', 'NodeJS', 'ngrx state management', 'Java', 'Spring framework', 'browser - based automation frameworks', 'Cypress', 'Selenium', 'Linux']

Soft Skills: ['communication', 'interpersonal skills']
